In [2]:
import requests
from copy import copy


def dfs(category_id):
    url = "https://api.itemscout.io/api/category/{0}/subcategories".format(category_id)
    response = requests.get(url = url)
    
    for d in response.json()['data']:

        d_id = d['id']
        d_name = d['name']
        d_is_leaf = d['is_leaf']
        
        if(d_is_leaf):
            leaf_id = copy(d_id)
            leaf_name = copy(d_name)
            leaf_category.append([leaf_id,leaf_name])
            
        else:
            dfs(d_id)
    
paths = []
leaf_category = []
starting_category = [1,2,3,4,5,8,9] #출산/육아(6) , 식품 제외(7)


first_category = [1,2,3,4,5]

#for category in first_category:
#    dfs(category)
dfs(1)


print(len(leaf_category))
#1 : 패션의류 --> 68개 
#2 : 패션잡화 --> 264개
#3 : 화장품/미용 --> 157개
#4 : 디지털/가전 --> 681개
#5 : 가구/인테리어 --> 354개
#6 : 출산/육아 (X)
#7 : 식품 (X)
#8 : 스포츠/레저 --> 카테고리 : 480개, 키워드 : 154207개 --> 필터 적용 : 20000개
#9 : 생활/건강 --> 카테고리 : 1227개, 키워드 : 465817개 --> 필터 적용 : 61751개

#app 1개당 기본 api 12500개 
#계정 1개당 app 10개, api 125000개

68


In [22]:
import json
import urllib.request
from collections import OrderedDict
from tqdm import notebook
from joblib import Parallel, delayed
from time import sleep
from config import *
from infsell import *



def extract_titles(category,client_auth):
    print(category)
    
    category_id = category[0]
    category_name = category[1]
    
    url_data = "https://api.itemscout.io/api/category/{0}/data".format(category_id)
    url_brands = "https://api.itemscout.io/api/category/{0}/brands".format(category_id)
    response_data = requests.post(url_data, json=PAYLOAD)
    response_brands = requests.get(url_brands)
    data = response_data.json()['data']['data']
    brands = response_brands.json()['data']
    brands = remove_a_word(brands)
    
    keywords,dict_clicks = get_keywords(data)
    keywords_nb = remove_brands(keywords,brands)
    
   
    
    for keyword in notebook.tqdm(keywords_nb[:5], desc = "키워드 진행도"):
        encText = urllib.parse.quote(keyword)
        url_items = "https://openapi.naver.com/v1/search/shop.json?display=80&query=" + encText # JSON 결과
        titles = []
        
        try:
            items = get_items(url_items,client_auth)
            
            for item in items:
                title = item['title'].replace('<b>','').replace('</b>','')
                titles.append(title)
            
            info_keyword={}
            info_keyword['category'] = category_name
            info_keyword['keyword'] = keyword
            info_keyword['titles'] = titles

            titles_data.append(info_keyword)

        except:
            print('error 발생, 키워드 : ', keyword)
            pass
            
       
        

#멀티코어 개수
num_core = 12

cnt_keyword=0
path_auth='auth_api.txt'
client_auth = get_auth_api(path_auth)
titles_data = []

with Parallel(n_jobs=num_core, require='sharedmem') as parallel:
    parallel(delayed(extract_titles)(category,client_auth)for category in notebook.tqdm(leaf_category[:5]))
    

json_titles_data = json.dumps(titles_data,indent=2)
print(json_titles_data)

  0%|          | 0/5 [00:00<?, ?it/s]

[660, '코디세트'][683, '티셔츠']

[684, '블라우스/셔츠']
[685, '니트/스웨터']
[686, '카디건']


키워드 진행도:   0%|          | 0/5 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/5 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/5 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/5 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/5 [00:00<?, ?it/s]

[
  {
    "category": "\ube14\ub77c\uc6b0\uc2a4/\uc154\uce20",
    "keyword": "\ub3c4\ud2b8\ube14\ub77c\uc6b0\uc2a4",
    "titles": [
      "\ub9ac\uc2a4\ud2b8 \uc26c\ud3f0 \ub3c4\ud2b8 \ube14\ub77c\uc6b0\uc2a4 TWWSTM62020",
      "JJ\uc9c0\uace0\ud2b8 JJ\uc9c0\uace0\ud2b8 JJ\uc9c0\uace0\ud2b8 \ubc30\uc0c9 \uce7c\ub77c \ub3c4\ud2b8 \ube14\ub77c\uc6b0\uc2a4 GN5P0BL54",
      "\ub4dc\ub12c \uc0e4\uc778\ub3c4\ud2b8\ube14\ub77c\uc6b0\uc2a4 DB11755A",
      "\uc5b4\uae68\uc154\ub9c1 \ub77c\uc6b4\ub4dc \ub3c4\ud2b8 \ube14\ub77c\uc6b0\uc2a4",
      "\ub370\ucf54\uc5ec\ub984 \uacbd\uc751 \uc568\ub9ac\uc2a4\ub3c4\ud2b8 \ube14\ub77c\uc6b0\uc2a4 FDBAC3751W",
      "\ub4c0\uc5d8 \ub3c4\ud2b8 \ud504\ub9b4\uc18c\ub9e4 \ube14\ub77c\uc6b0\uc2a4 CR L232MSC035",
      "\uc26c\uc988\ubbf8\uc2a4 \ub3c4\ud2b8 \ud0c0\uc774 \ube14\ub77c\uc6b0\uc2a4 SWWBLM21050",
      "\ub3c5\uc77c \uc790\ub77c ZARA \ub7ec\ud50c \ub3c4\ud2b8 \ub9dd\uc0ac \ube14\ub77c\uc6b0\uc2a4 245646",
      "\ud3ec\ub77c\ub9ac \ud3ec\ub77

In [27]:
with open('test.json', 'w', encoding='uft-8') as f:
    f.write(json.dumps(titles_data, ensure_ascii=False))


LookupError: unknown encoding: uft-8